In [ ]:
# wordcloud and boost specific words?
# statsmodels, ordinary least squares, pvalue


In [7]:
import pandas as pd
import numpy as np
from string import punctuation
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE


In [8]:
fname = "../data/S1_8Sharktankpitchesdeals.csv"
data = pd.read_csv(fname)
#valuation is a component of the amoutn asked for and the stake offered therefor doesn't need to be included in the machine learning
data = data.drop(columns=["Valuation"])
data


,Season_Epi_code,Pitched_Business_Identifier,Amount_Asked_For,Exchange_For_Stake,Gender,Category,Pitched_Business_Desc,Deal_Status,Kevin O'Leary,Lori Greiner,Mark Cuban,Robert Herjavec,Daymond John,Barbara Corcoran,Others,Deal_Shark 1,Deal_Shark 2,Deal_Shark 3,Deal_Shark 4,Deal_Shark 5
0,101,Ava the Elephant,50000,0.15,Female,Health / Wellness,"(Emmy the Elephant during show, trademarked a...",1,0,0,0,0,0,1,0,Barbara Corcoran,NaN,NaN,NaN,NaN
1,101,College Foxes Packing Boxes,250000,0.25,Male,Lifestyle / Home,a packing and organizing service based on an a...,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2,101,Ionic Ear,1000000,0.15,Male,Software / Tech,an implantable Bluetooth device requiring surg...,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
3,101,Mr. Tod's Pie Factory,460000,0.10,Male,Food and Beverage,a pie company,1,0,0,0,0,1,1,0,Barbara Corcoran,Daymond John,NaN,NaN,NaN
4,101,Wispots,1200000,0.10,Male,Business Services,an electronic hand-held device for waiting roo...,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,824,Wallet Buckle,500000,0.10,Male,Fashion / Beauty,a belt buckle that holds credit cards . The Wa...,1,0,0,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
700,826,Bridal Buddy,75000,0.10,Female,Fashion / Beauty,a functional slip worn under a wedding gown th...,1,1,1,0,0,0,0,0,Kevin O'Leary,Lori Greiner,NaN,NaN,NaN
701,826,Laid Brand,30000,0.30,Male,Fashion / Beauty,hair-care products made with pheromones . Laid...,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
702,826,Rocketbook,400000,0.10,Male,Software / Tech,a notebook that can scan contents to cloud ser...,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN


In [9]:
data2 = pd.DataFrame(columns=data.columns)
offset = 0;

for i, j in data.iterrows():
    data2 = data2.append(data.loc[i].to_dict(), ignore_index=True)
    
    if not pd.isna(data["Deal_Shark 2"][i]):
        data2 = data2.append(data.loc[i].to_dict(), ignore_index=True)
        offset+=1
        data2["Deal_Shark 1"][offset+i] = data.loc[i]["Deal_Shark 2"]
    if not pd.isna(data["Deal_Shark 3"][i]):
        data2 = data2.append(data.loc[i].to_dict(), ignore_index=True)
        offset+=1
        data2["Deal_Shark 1"][offset+i] = data.loc[i]["Deal_Shark 3"]
    if not pd.isna(data["Deal_Shark 4"][i]):
        data2 = data2.append(data.loc[i].to_dict(), ignore_index=True)
        offset+=1
        data2["Deal_Shark 1"][offset+i] = data.loc[i]["Deal_Shark 4"]
    if not pd.isna(data["Deal_Shark 5"][i]):
        data2 = data2.append(data.loc[i].to_dict(), ignore_index=True)
        offset+=1
        data2["Deal_Shark 1"][offset+i] = data.loc[i]["Deal_Shark 5"]


data2 = data2.drop(columns=["Deal_Shark 2","Deal_Shark 3","Deal_Shark 4","Deal_Shark 5"])
data2

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\

,Season_Epi_code,Pitched_Business_Identifier,Amount_Asked_For,Exchange_For_Stake,Gender,Category,Pitched_Business_Desc,Deal_Status,Kevin O'Leary,Lori Greiner,Mark Cuban,Robert Herjavec,Daymond John,Barbara Corcoran,Others,Deal_Shark 1
0,101,Ava the Elephant,50000,0.15,Female,Health / Wellness,"(Emmy the Elephant during show, trademarked a...",1,0,0,0,0,0,1,0,Barbara Corcoran
1,101,College Foxes Packing Boxes,250000,0.25,Male,Lifestyle / Home,a packing and organizing service based on an a...,0,0,0,0,0,0,0,0,NaN
2,101,Ionic Ear,1000000,0.15,Male,Software / Tech,an implantable Bluetooth device requiring surg...,0,0,0,0,0,0,0,0,NaN
3,101,Mr. Tod's Pie Factory,460000,0.10,Male,Food and Beverage,a pie company,1,0,0,0,0,1,1,0,Barbara Corcoran
4,101,Mr. Tod's Pie Factory,460000,0.10,Male,Food and Beverage,a pie company,1,0,0,0,0,1,1,0,Daymond John
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,826,Bridal Buddy,75000,0.10,Female,Fashion / Beauty,a functional slip worn under a wedding gown th...,1,1,1,0,0,0,0,0,Kevin O'Leary
806,826,Bridal Buddy,75000,0.10,Female,Fashion / Beauty,a functional slip worn under a wedding gown th...,1,1,1,0,0,0,0,0,Lori Greiner
807,826,Laid Brand,30000,0.30,Male,Fashion / Beauty,hair-care products made with pheromones . Laid...,0,0,0,0,0,0,0,0,NaN
808,826,Rocketbook,400000,0.10,Male,Software / Tech,a notebook that can scan contents to cloud ser...,0,0,0,0,0,0,0,0,NaN


In [10]:
data = data.infer_objects()
data.dtypes

Season_Epi_code                  int64
Pitched_Business_Identifier     object
Amount_Asked_For                 int64
Exchange_For_Stake             float64
Gender                          object
Category                        object
Pitched_Business_Desc           object
Deal_Status                      int64
Kevin O'Leary                    int64
Lori Greiner                     int64
Mark Cuban                       int64
Robert Herjavec                  int64
Daymond John                     int64
Barbara Corcoran                 int64
Others                           int64
Deal_Shark 1                    object
Deal_Shark 2                    object
Deal_Shark 3                    object
Deal_Shark 4                    object
Deal_Shark 5                    object
dtype: object

In [11]:
data2 = data2.infer_objects()
data2.dtypes

Season_Epi_code                  int64
Pitched_Business_Identifier     object
Amount_Asked_For                 int64
Exchange_For_Stake             float64
Gender                          object
Category                        object
Pitched_Business_Desc           object
Deal_Status                      int64
Kevin O'Leary                    int64
Lori Greiner                     int64
Mark Cuban                       int64
Robert Herjavec                  int64
Daymond John                     int64
Barbara Corcoran                 int64
Others                           int64
Deal_Shark 1                    object
dtype: object

In [12]:
# Choose model output

# "Deal_Status"
# "Deal_Shark 1"

# "Kevin O'Leary"
# "Barbara Corcoran"
# "Robert Herjavec"
# "Daymond John"
# "Lori Greiner"
# "Mark Cuban"
# "Others"
shark_to_model = "Deal_Status"

In [13]:
# drop rows where shark_to_model is Nan
data=data.dropna(subset=[shark_to_model])
data2=data2.dropna(subset=[shark_to_model])
# .reset_index(drop=True)
data

,Season_Epi_code,Pitched_Business_Identifier,Amount_Asked_For,Exchange_For_Stake,Gender,Category,Pitched_Business_Desc,Deal_Status,Kevin O'Leary,Lori Greiner,Mark Cuban,Robert Herjavec,Daymond John,Barbara Corcoran,Others,Deal_Shark 1,Deal_Shark 2,Deal_Shark 3,Deal_Shark 4,Deal_Shark 5
0,101,Ava the Elephant,50000,0.15,Female,Health / Wellness,"(Emmy the Elephant during show, trademarked a...",1,0,0,0,0,0,1,0,Barbara Corcoran,NaN,NaN,NaN,NaN
1,101,College Foxes Packing Boxes,250000,0.25,Male,Lifestyle / Home,a packing and organizing service based on an a...,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2,101,Ionic Ear,1000000,0.15,Male,Software / Tech,an implantable Bluetooth device requiring surg...,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
3,101,Mr. Tod's Pie Factory,460000,0.10,Male,Food and Beverage,a pie company,1,0,0,0,0,1,1,0,Barbara Corcoran,Daymond John,NaN,NaN,NaN
4,101,Wispots,1200000,0.10,Male,Business Services,an electronic hand-held device for waiting roo...,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,824,Wallet Buckle,500000,0.10,Male,Fashion / Beauty,a belt buckle that holds credit cards . The Wa...,1,0,0,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
700,826,Bridal Buddy,75000,0.10,Female,Fashion / Beauty,a functional slip worn under a wedding gown th...,1,1,1,0,0,0,0,0,Kevin O'Leary,Lori Greiner,NaN,NaN,NaN
701,826,Laid Brand,30000,0.30,Male,Fashion / Beauty,hair-care products made with pheromones . Laid...,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
702,826,Rocketbook,400000,0.10,Male,Software / Tech,a notebook that can scan contents to cloud ser...,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN


In [14]:
# Custom tokenizer from class activity 22-1-4
stopwords = stopwords.words( 'english' ) + list(punctuation)
stemmer = PorterStemmer()
# Stemming
punc_list = list(punctuation)
def special_remove(word):
    if len(word)>2:
        return False
    for c in word:
        if c in punc_list:
            return True
    return False

# custom function that overrides default token generation
def custom_tokenizer(text):
    text = text.lower()
    words = nltk.word_tokenize(text)
    words = [stemmer.stem(w) for w in words if w not in stopwords+["..."]]
    # further remove words with a special char
    words = [w for w in words if not special_remove(w)]
    return words

activity_vectorizer = TfidfVectorizer(ngram_range=(1,2)
                            ,tokenizer=custom_tokenizer 
                            ,stop_words='english'
                            # ,min_df=2
                            ,strip_accents='unicode'
                            )

activity_vectors = activity_vectorizer.fit_transform(data["Pitched_Business_Desc"])
activity_df = pd.DataFrame(activity_vectors.toarray(), columns=activity_vectorizer.get_feature_names())
activity_df

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'thu', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


,-not,-not simpli,"1,2pee",10,10 cool,10 day,10 minut,10 net,10 spandex,"10,000",...,“ total,“ uv,“ wrap,“ year,“ yuk,”,” day,” peopl,” potato,” prevent
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
activity_vectors2 = activity_vectorizer.fit_transform(data2["Pitched_Business_Desc"])
activity_df2 = pd.DataFrame(activity_vectors2.toarray(), columns=activity_vectorizer.get_feature_names())
activity_df2

,-not,-not simpli,"1,2pee",10,10 cool,10 day,10 minut,10 net,10 spandex,"10,000",...,“ total,“ uv,“ wrap,“ year,“ yuk,”,” day,” peopl,” potato,” prevent
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
807,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
808,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
activity_vectorizer.get_feature_names()

['-not',
 '-not simpli',
 '1,2pee',
 '10',
 '10 cool',
 '10 day',
 '10 minut',
 '10 net',
 '10 spandex',
 '10,000',
 '10,000 run',
 '10-30',
 '10-30 minut',
 '100',
 '100 built-in',
 '100 calori',
 '100 food',
 '100 guarante',
 '100 mile',
 '100 natur',
 '100 recycl',
 '100 time',
 '100 zinc',
 '100-calori',
 '100-calori pint',
 '11',
 '11 year',
 '12',
 '12 fashion',
 '12 pack',
 '123',
 '123 plush',
 '15',
 '15 flavor',
 '15 throw',
 '15 year',
 '15/30',
 '15/30 spf',
 '150',
 '150 90',
 '2',
 '2 ipad',
 '2 type',
 '2 week',
 '2-5',
 '2-5 age',
 '2-in-1',
 '2-in-1 spray',
 '20',
 '20 week',
 '20-30',
 '20-30 second',
 '2009',
 '2009 11',
 '2012',
 '2012 beardbrand',
 '2012 pipcorn',
 '20mph',
 '20mph 30mph',
 '21st',
 '21st centuri',
 '23',
 '23 ounc',
 '24',
 '24 mph',
 '2400',
 '2400 expert',
 '2400 sat',
 '24—24',
 '24—24 board',
 '26',
 '26 pound',
 '26g',
 '26g protein',
 '270',
 '270 shoe',
 '2700',
 '2700 diaper',
 '3',
 '3 design',
 '3 ipad',
 '3 pack',
 '3-d',
 '3-d design',

In [17]:
# Choose features

# To include Pitch, comment in next line, and comment out the following line
# To exclude Pitch, comment out next line, and comment in the following line

selected_features = activity_df
selected_features2 = activity_df2
# selected_features = pd.DataFrame([])

selected_features["Gender"]=data["Gender"]
selected_features["Category"]=data["Category"]
selected_features["Amount_Asked_For"]=data["Amount_Asked_For"]
selected_features["Exchange_For_Stake"]=data["Exchange_For_Stake"]
selected_features

,-not,-not simpli,"1,2pee",10,10 cool,10 day,10 minut,10 net,10 spandex,"10,000",...,“ yuk,”,” day,” peopl,” potato,” prevent,Gender,Category,Amount_Asked_For,Exchange_For_Stake
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Female,Health / Wellness,50000,0.15
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Lifestyle / Home,250000,0.25
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Software / Tech,1000000,0.15
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Food and Beverage,460000,0.10
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Business Services,1200000,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Fashion / Beauty,500000,0.10
700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Female,Fashion / Beauty,75000,0.10
701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Fashion / Beauty,30000,0.30
702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Software / Tech,400000,0.10


In [18]:
selected_features2["Gender"]=data2["Gender"]
selected_features2["Category"]=data2["Category"]
selected_features2["Amount_Asked_For"]=data2["Amount_Asked_For"]
selected_features2["Exchange_For_Stake"]=data2["Exchange_For_Stake"]
selected_features2

,-not,-not simpli,"1,2pee",10,10 cool,10 day,10 minut,10 net,10 spandex,"10,000",...,“ yuk,”,” day,” peopl,” potato,” prevent,Gender,Category,Amount_Asked_For,Exchange_For_Stake
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Female,Health / Wellness,50000,0.15
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Lifestyle / Home,250000,0.25
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Software / Tech,1000000,0.15
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Food and Beverage,460000,0.10
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Food and Beverage,460000,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Female,Fashion / Beauty,75000,0.10
806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Female,Fashion / Beauty,75000,0.10
807,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Fashion / Beauty,30000,0.30
808,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Software / Tech,400000,0.10


In [19]:
selected_features = pd.get_dummies(selected_features)
selected_features

,-not,-not simpli,"1,2pee",10,10 cool,10 day,10 minut,10 net,10 spandex,"10,000",...,Category_Fitness / Sports / Outdoor,Category_Food and Beverage,Category_Green / Clean Tech,Category_Health / Wellness,Category_Lifestyle / Home,Category_Media / Entertainment,Category_Pet Products,Category_Software / Tech,Category_Travel,Category_Uncertain / Other
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0


In [20]:
selected_features2 = pd.get_dummies(selected_features2)
selected_features2



,-not,-not simpli,"1,2pee",10,10 cool,10 day,10 minut,10 net,10 spandex,"10,000",...,Category_Fitness / Sports / Outdoor,Category_Food and Beverage,Category_Green / Clean Tech,Category_Health / Wellness,Category_Lifestyle / Home,Category_Media / Entertainment,Category_Pet Products,Category_Software / Tech,Category_Travel,Category_Uncertain / Other
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
807,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
808,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0


In [21]:
selected_features.columns

Index(['-not', '-not simpli', '1,2pee', '10', '10 cool', '10 day', '10 minut',
       '10 net', '10 spandex', '10,000',
       ...
       'Category_Fitness / Sports / Outdoor', 'Category_Food and Beverage',
       'Category_Green / Clean Tech', 'Category_Health / Wellness',
       'Category_Lifestyle / Home', 'Category_Media / Entertainment',
       'Category_Pet Products', 'Category_Software / Tech', 'Category_Travel',
       'Category_Uncertain / Other'],
      dtype='object', length=13189)

In [22]:
model = MultinomialNB()

X_smote, y_smote = SMOTE().fit_sample(selected_features, data[[shark_to_model]].values.ravel())
X = X_smote
y = y_smote

# X = selected_features
# y = data[[shark_to_model]].values.ravel()



print(X.shape)
print(y.shape)

(764, 13189)
(764,)


In [23]:
model2 = MultinomialNB()

X_smote2, y_smote2 = SMOTE().fit_sample(selected_features2, data2[[shark_to_model]].values.ravel())
X2 = X_smote2
y2 = y_smote2

# X2 = selected_features2
# y2 = data2[[shark_to_model]].values.ravel()



print(X2.shape)
print(y2.shape)

(976, 13189)
(976,)


In [24]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=f_classif, k=6)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfpvalues = pd.DataFrame(fit.pvalues_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores,dfpvalues],axis=1)
featureScores.columns = ['Specs','Score','PValue']  #naming the dataframe columns
print(featureScores.nlargest(20,'Score'))  #print 10 best features

                           Specs      Score    PValue
3663                        easi  10.176195  0.001481
6249                         kid   9.765298  0.001846
13171         Exchange_For_Stake   9.257249  0.002426
5276                        hand   8.459366  0.003737
7096                      market   7.740102  0.005534
12344                    varieti   7.605070  0.005960
5774                         ice   6.576270  0.010525
8898                       price   6.385572  0.011706
3353                      differ   6.382629  0.011726
3005                         cut   6.251988  0.012614
9896                      school   6.018651  0.014378
12502                       wait   5.985655  0.014648
7379                        miss   5.937758  0.015048
13183  Category_Lifestyle / Home   5.870416  0.015629
12732                      wheel   5.586780  0.018347
13010                       year   5.522218  0.019031
1155                      better   5.484172  0.019446
8328                     per

In [25]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=6)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfpvalues = pd.DataFrame(fit.pvalues_)
dfcolumns = pd.DataFrame(X2.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores,dfpvalues],axis=1)
featureScores.columns = ['Specs','Score','PValue']  #naming the dataframe columns
print(featureScores.nlargest(20,'Score'))  #print 10 best features

                                     Specs          Score    PValue
13170                     Amount_Asked_For  108280.396155  0.000000
13183            Category_Lifestyle / Home       5.038095  0.024796
13176           Category_Business Services       3.000000  0.083265
13179  Category_Fitness / Sports / Outdoor       2.800000  0.094264
9808                                sandal       1.970229  0.160424
13177        Category_Children / Education       1.894737  0.168669
13174                         Gender_Mixed       1.814815  0.177932
3663                                  easi       1.642885  0.199930
5774                                   ice       1.603601  0.205394
6173                               jewelri       1.594189  0.206729
3065                                  date       1.533960  0.215519
5276                                  hand       1.497396  0.221072
12820                                 wipe       1.455546  0.227640
6163                                  jean      

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [27]:
model.fit(X_train, y_train.ravel())

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [28]:
y_pred = model.predict(X_test)

In [29]:
pd.DataFrame({"actual": y_test.reshape(-1), "prediction": y_pred.reshape(-1)})

,actual,prediction
0,0,1
1,1,1
2,0,0
3,0,1
4,1,1
...,...,...
186,0,1
187,0,1
188,0,1
189,1,1


In [30]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5654450261780105

In [31]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, random_state=42)

In [32]:
model2.fit(X_train2, y_train2.ravel())

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [33]:
y_pred2 = model2.predict(X_test2)

In [34]:
pd.DataFrame({"actual": y_test2.reshape(-1), "prediction": y_pred2.reshape(-1)})

,actual,prediction
0,0,0
1,1,0
2,1,0
3,0,0
4,1,1
...,...,...
239,1,1
240,1,0
241,0,0
242,1,0


In [35]:
accuracy_score(y_test2,y_pred2)

0.6065573770491803

In [36]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
param_grid = {'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}
scores = ['precision', 'recall']
model3 = MultinomialNB()

In [37]:
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        model3, param_grid, scoring='%s_macro' % score,verbose=3)
    clf.fit(X_train2, y_train2)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test2, clf.predict(X_test2)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] alpha=1e-05 .....................................................
[CV] ......................... alpha=1e-05, score=0.703, total=   0.1s
[CV] alpha=1e-05 .....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ......................... alpha=1e-05, score=0.694, total=   0.2s
[CV] alpha=1e-05 .....................................................
[CV] ......................... alpha=1e-05, score=0.726, total=   0.1s
[CV] alpha=1e-05 .....................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV] ......................... alpha=1e-05, score=0.788, total=   0.1s
[CV] alpha=1e-05 .....................................................
[CV] ......................... alpha=1e-05, score=0.733, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.703, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.700, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.733, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.781, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.719, total=   0.1s
[CV] alpha=0.001 .....................................................
[CV] .

...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] ......................... alpha=1e-05, score=0.700, total=   0.1s
[CV] alpha=1e-05 .....................................................
[CV] ......................... alpha=1e-05, score=0.686, total=   0.1s
[CV] alpha=1e-05 .....................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV] ......................... alpha=1e-05, score=0.726, total=   0.1s
[CV] alpha=1e-05 .....................................................
[CV] ......................... alpha=1e-05, score=0.788, total=   0.1s
[CV] alpha=1e-05 .....................................................
[CV] ......................... alpha=1e-05, score=0.733, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.700, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.693, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.733, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.781, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] .

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    5.8s finished


In [38]:
# Import ML models from sklearn
from sklearn.linear_model import LogisticRegression # Regression classifier
from sklearn.tree import DecisionTreeClassifier # Decision Tree classifier
from sklearn import svm # Support Vector Machine
from sklearn.linear_model import SGDClassifier # Stochastic Gradient Descent Classifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier # Random Forest and Gradient Boosting Classifier
from sklearn.naive_bayes import MultinomialNB # Naive Bayes Classifier 
from sklearn.metrics import f1_score, accuracy_score, recall_score, confusion_matrix # Some metrics to check the performance of the models

In [39]:
# Setting parameters for each algorithm - these are tunable to achieve max accuracy

Classifiers = {'LR':LogisticRegression(random_state=10,C=5,max_iter=300, solver='lbfgs')
               ,'DTC':DecisionTreeClassifier(random_state=10,min_samples_leaf=2)
               ,'RF':RandomForestClassifier(random_state=10,n_estimators=100,n_jobs=-1)
               # ,'GBC':GradientBoostingClassifier(random_state=10,n_estimators=400,learning_rate=0.2)
               ,'SGD':SGDClassifier(loss="hinge", penalty="l2")
               # ,'SVM':svm.SVC(kernel='linear', C=0.1)
               ,'NB':MultinomialNB(alpha=.0001)
               }

In [40]:
Classifiers.keys()

dict_keys(['LR', 'DTC', 'RF', 'SGD', 'NB'])

In [41]:
# Create a pipeline so you can reuse the code

def ML_Pipeline(clf_name):
    clf = Classifiers[clf_name]
    fit = clf.fit(X_train, y_train.ravel())
    y_pred = clf.predict(X_test)
    Accuracy = accuracy_score(y_test,y_pred)
    Precision = metrics.precision_score(y_test,y_pred)
    Recall = recall_score(y_test,y_pred)
    Confusion_matrix = confusion_matrix(y_test,y_pred)
    print('Classifier = '+(clf_name))
    print('Accuracy = '+str(Accuracy))
    print('Precision = '+str(Precision))
    print('Recall = '+str(Recall))
    print(Confusion_matrix)
    print('==='*20) 
    return clf

In [42]:
# Create a pipeline so you can reuse the code

def ML_Pipeline2(clf_name):
    clf = Classifiers[clf_name]
    fit = clf.fit(X_train2, y_train2.ravel())
    y_pred = clf.predict(X_test2)
    F1 = f1_score(y_test2, y_pred, zero_division=1)
    Accuracy = accuracy_score(y_test2,y_pred)
    Precision = metrics.precision_score(y_test2,y_pred)
    Recall = recall_score(y_test2,y_pred)
    Confusion_matrix = confusion_matrix(y_test2,y_pred)
    print('Classifier = '+(clf_name))
    print('F1 = '+str(F1))
    print('Accuracy = '+str(Accuracy))
    print('Precision = '+str(Precision))
    print('Recall = '+str(Recall))
    print(Confusion_matrix)
    print('==='*20) 
    return clf

In [43]:
clf_dict = []
for clf_name in Classifiers.keys():
    clf = Classifiers[clf_name]
    fit = clf.fit(X_train, y_train.ravel())
    y_pred = clf.predict(X_test)
    Accuracy = accuracy_score(y_test,y_pred)
    Precision = metrics.precision_score(y_test,y_pred)
    Recall = recall_score(y_test,y_pred)
    Confusion_matrix = confusion_matrix(y_test,y_pred)
    clf_dict.append({'Classifier': clf_name, 'Accuracy': Accuracy, 'Precision': Precision, 'Recall': Recall})
    print('Classifier = '+(clf_name))
    print('Accuracy = '+str(Accuracy))
    print('Precision = '+str(Precision))
    print('Recall = '+str(Recall))
    print(Confusion_matrix)
    print('==='*20)


Classifier = LR
Accuracy = 0.4869109947643979
Precision = 0.4869109947643979
Recall = 1.0
[[ 0 98]
 [ 0 93]]
Classifier = DTC
Accuracy = 0.5287958115183246
Precision = 0.5205479452054794
Recall = 0.40860215053763443
[[63 35]
 [55 38]]
Classifier = RF
Accuracy = 0.6387434554973822
Precision = 0.6428571428571429
Recall = 0.5806451612903226
[[68 30]
 [39 54]]
Classifier = SGD
Accuracy = 0.4869109947643979
Precision = 0.4869109947643979
Recall = 1.0
[[ 0 98]
 [ 0 93]]
Classifier = NB
Accuracy = 0.6020942408376964
Precision = 0.5876288659793815
Recall = 0.6129032258064516
[[58 40]
 [36 57]]


In [44]:
clf_dict2 = []
for clf_name in Classifiers.keys():
    clf = Classifiers[clf_name]
    fit = clf.fit(X_train2, y_train2.ravel())
    y_pred = clf.predict(X_test2)
    F1 = f1_score(y_test2, y_pred, zero_division=1)
    Accuracy = accuracy_score(y_test2,y_pred)
    Precision = metrics.precision_score(y_test2,y_pred)
    Recall = recall_score(y_test2,y_pred)
    Confusion_matrix = confusion_matrix(y_test2,y_pred)
    clf_dict2.append({'Classifier': clf_name, 'F1': F1, 'Accuracy': Accuracy, 'Precision': Precision, 'Recall': Recall})
    print('Classifier = '+(clf_name))
    print('F1 = '+str(F1))
    print('Accuracy = '+str(Accuracy))
    print('Precision = '+str(Precision))
    print('Recall = '+str(Recall))
    print(Confusion_matrix)
    print('==='*20)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classifier = LR
F1 = 0.0
Accuracy = 0.4959016393442623
Precision = 0.0
Recall = 0.0
[[121   0]
 [123   0]]
Classifier = DTC
F1 = 0.6244725738396625
Accuracy = 0.6352459016393442
Precision = 0.6491228070175439
Recall = 0.6016260162601627
[[81 40]
 [49 74]]
Classifier = RF
F1 = 0.7307692307692308
Accuracy = 0.7131147540983607
Precision = 0.6934306569343066
Recall = 0.7723577235772358
[[79 42]
 [28 95]]
Classifier = SGD
F1 = 0.0
Accuracy = 0.4959016393442623
Precision = 0.0
Recall = 0.0
[[121   0]
 [123   0]]
Classifier = NB
F1 = 0.7478991596638654
Accuracy = 0.7540983606557377
Precision = 0.7739130434782608
Recall = 0.7235772357723578
[[95 26]
 [34 89]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [45]:
clf_df = pd.DataFrame(clf_dict2)
clf_df

,Classifier,F1,Accuracy,Precision,Recall
0,LR,0.000000,0.495902,0.000000,0.000000
1,DTC,0.624473,0.635246,0.649123,0.601626
2,RF,0.730769,0.713115,0.693431,0.772358
3,SGD,0.000000,0.495902,0.000000,0.000000
4,NB,0.747899,0.754098,0.773913,0.723577


In [46]:
clf_df['Accuracy'].max()

0.7540983606557377

In [47]:
best_clf = clf_df.loc[clf_df['Accuracy'].idxmax(),'Classifier']
best_clf

'NB'

In [48]:
best_model = ML_Pipeline2(best_clf)
best_model


Classifier = NB
F1 = 0.7478991596638654
Accuracy = 0.7540983606557377
Precision = 0.7739130434782608
Recall = 0.7235772357723578
[[95 26]
 [34 89]]


MultinomialNB(alpha=0.0001, class_prior=None, fit_prior=True)

In [50]:
import joblib
joblib.dump(best_model, f"{shark_to_model}_model.pkl")

['Deal_Status_model.pkl']

In [51]:
import pickle
pickle.dump(selected_features.columns,open(f"{shark_to_model}_vocab.pkl","wb"))